In [ ]:
import importlib

In [2]:
from oi_bulk_get_fns import get_all_openinsider_chunks,openinsider_url
import pandas as pd

http://openinsider.com/screener?s=&o=&pl=&ph=&ll=&lh=&fd=-1&td=0&tdr=&fdlyl=&fdlyh=&daysago=&xp=1&vl=&vh=&ocl=&och=&sic1=-1&sicl=100&sich=9999&grp=0&nfl=&nfh=&nil=&nih=&nol=&noh=&v2l=&v2h=&oc2l=&oc2h=&sortcol=0&cnt=1000&page=1&fdr=05%2F01%2F2022+-+05%2F10%2F2023


In [ ]:
#IFF need to re-pull from OI:
start = pd.Timestamp('2019-01-01')
end = pd.Timestamp('2025-08-06')
get_all_openinsider_chunks(start,end,chunk_size=1000,window_days=45,overlap_days=0)

In [ ]:
filename = 'oi_rawpull_ymd_2019_01_01_2025_08_06_incl.csv'
oipmega1 = pd.read_csv(filename, usecols=lambda col: col != pd.read_csv(filename, nrows=0).columns[0])

In [11]:
print(oipmega1.head())
print('number of rows:', len(oipmega1))
tickers = oipmega1["ticker"].unique().tolist()
print(oipmega1.columns)

           filing_date  trade_date ticker            company_name  \
0  2025-08-06 21:18:06  2025-08-05    WSC  Willscot Holdings Corp   
1  2025-08-06 20:48:14  2025-08-04   PRLB          Proto Labs Inc   
2  2025-08-06 19:47:30  2025-08-04   RVSB   Riverview Bancorp Inc   
3  2025-08-06 19:21:21  2025-08-04   TBBK           Bancorp, Inc.   
4  2025-08-06 18:43:37  2025-08-06   ASUR      Asure Software Inc   

         insider_name      title    trade_type   qty    owned     value  \
0  Soultz Bradley Lee        CEO  P - Purchase  5000   739418  125750.0   
1      Krishna Suresh  Pres, CEO  P - Purchase  1725    18021   75055.0   
2        Benke Robert   EVP, CCO  P - Purchase  4000    45578   19400.0   
3        Cohn Matthew        Dir  P - Purchase   550   227244   34931.0   
4      Goepel Patrick   COB, CEO  P - Purchase  5250  1307622   42840.0   

   insider_price  d_own_plus%_isnew  d_own_plus%  
0          25.15                  0          1.0  
1          43.51                

In [13]:
def print_top_volume_dates(df, date_col='filing_date', top_n=10):
    # Extract just the date (not time)
    df_dates = df.copy()
    df_dates['date_only'] = pd.to_datetime(df_dates[date_col]).dt.date
    # Group by date and count
    date_counts = df_dates.groupby('date_only').size()
    # Sort in descending order and take top N
    top_dates = date_counts.sort_values(ascending=False).head(top_n)
    # Print
    print(f"Top {top_n} dates by #filings (descending):")
    for d, count in top_dates.items():
        print(f"{d}: {count} filings")
    return top_dates

# Usage:
print_top_volume_dates(oipmega1, top_n=20)



Top 20 dates by #filings (descending):
2020-03-11: 557 filings
2020-03-16: 418 filings
2020-03-13: 408 filings
2020-03-12: 365 filings
2020-03-17: 303 filings
2020-03-10: 284 filings
2019-01-02: 280 filings
2019-03-05: 276 filings
2020-03-18: 269 filings
2023-03-15: 258 filings
2020-03-19: 238 filings
2020-03-09: 231 filings
2019-07-03: 219 filings
2021-08-18: 217 filings
2020-03-20: 214 filings
2022-06-21: 214 filings
2019-06-04: 207 filings
2020-03-05: 203 filings
2019-08-15: 196 filings
2023-05-04: 194 filings


date_only
2020-03-11    557
2020-03-16    418
2020-03-13    408
2020-03-12    365
2020-03-17    303
2020-03-10    284
2019-01-02    280
2019-03-05    276
2020-03-18    269
2023-03-15    258
2020-03-19    238
2020-03-09    231
2019-07-03    219
2021-08-18    217
2020-03-20    214
2022-06-21    214
2019-06-04    207
2020-03-05    203
2019-08-15    196
2023-05-04    194
dtype: int64

In [15]:
#YF PRICE/vol DATA ATTEMPT 1
from add_relative_prices2 import add_prices_to_oip_precise
window = [1,2,3,4,5,6,7,8,9,10,15,20,35,60,90,130,180,245]
oip_w_prices, error_dict = add_prices_to_oip_precise(oipmega1, fwindow=window, bwindow=window)
with pd.option_context('display.max_columns', None):
    #print(df.head())
    print(oip_w_prices.head(3))

1 Failed download:
['BCAR']: YFPricesMissingError('possibly delisted; no price data found  (1d 2024-03-23 00:00:00 -> 2026-12-18 00:00:00)')
1 Failed download:
['APAD']: YFPricesMissingError('possibly delisted; no price data found  (1d 2024-03-18 00:00:00 -> 2026-12-13 00:00:00)')
1 Failed download:
['NMP']: YFPricesMissingError('possibly delisted; no price data found  (1d 2024-02-18 00:00:00 -> 2026-11-22 00:00:00)')
1 Failed download:
['NRX']: YFPricesMissingError('possibly delisted; no price data found  (1d 2023-12-10 00:00:00 -> 2026-11-15 00:00:00)')

1 Failed download:
['FIGX']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['PPTIN']: YFTzMissingError('possibly delisted; no timezone found')
1 Failed download:
['VREO']: YFTzMissingError('possibly delisted; no timezone found')
1 Failed download:
['PTN']: YFTzMissingError('possibly delisted; no timezone found')
1 Failed download:
['SWPR']: YFTzMissingError('possibly delisted; no timezone found')
1 Fail

  filing_date  trade_date ticker            company_name        insider_name  \
0  2025-08-06  2025-08-05    WSC  Willscot Holdings Corp  Soultz Bradley Lee   
1  2025-08-06  2025-08-04   PRLB          Proto Labs Inc      Krishna Suresh   
2  2025-08-06  2025-08-04   RVSB   Riverview Bancorp Inc        Benke Robert   

       title    trade_type   qty   owned     value  insider_price  \
0        CEO  P - Purchase  5000  739418  125750.0          25.15   
1  Pres, CEO  P - Purchase  1725   18021   75055.0          43.51   
2   EVP, CCO  P - Purchase  4000   45578   19400.0           4.85   

   d_own_plus%_isnew  d_own_plus% mebuydate p_p1_td v_p1_td p_p2_td v_p2_td  \
0                  0          1.0       NaT    None    None    None    None   
1                  0         11.0       NaT    None    None    None    None   
2                  0         10.0       NaT    None    None    None    None   

  p_p3_td v_p3_td p_p4_td v_p4_td p_p5_td v_p5_td p_p6_td v_p6_td p_p7_td  \
0    Non

In [16]:
#save first price/vol pull checkpoint
oip_w_prices.to_csv('oipmega_post_yf1.csv',index=False)

In [20]:
#SEPARATE TRAIN/TEST/VALIDATION (all are inclusive) - these refer to filing_Dates !!!
#train: 1/1/2019 - 4/1/2025 oip1
#test: 4/2/2025 - 7/6/2025 oip2
#validation: simulate portfolio from then on? oip3
oip_w_prices['filing_date'] = pd.to_datetime(oipmega1['filing_date'])

# Define date boundaries
train_start = pd.Timestamp('2019-01-01')
train_end   = pd.Timestamp('2025-04-01')  # inclusive
test_start  = pd.Timestamp('2025-04-02')
test_end    = pd.Timestamp('2025-07-06')  # inclusive

# Create splits
oip1 = oip_w_prices[
    (oip_w_prices['filing_date'] >= train_start) &
    (oip_w_prices['filing_date'] <= train_end)
]

oip2 = oip_w_prices[
    (oip_w_prices['filing_date'] >= test_start) &
    (oip_w_prices['filing_date'] <= test_end)
]

print(f"Train rows: {len(oip1)}")
print(f"Test rows: {len(oip2)}")

Train rows: 89524
Test rows: 3015


In [ ]:
# count = 0
# for k, v in error_dict.items():
#     print(k, v)
#     count += 1
#     if count == 10:
#         break


WSC {0: {'mebuydate': 'No trading day after 2025-08-06 00:00:00'}, 852: {'p_p35_td': 'Not enough future data for 35 trading days forward', 'v_p35_td': 'Not enough future data for 35 trading days forward', 'p_p60_td': 'Not enough future data for 60 trading days forward', 'v_p60_td': 'Not enough future data for 60 trading days forward', 'p_p90_td': 'Not enough future data for 90 trading days forward', 'v_p90_td': 'Not enough future data for 90 trading days forward', 'p_p130_td': 'Not enough future data for 130 trading days forward', 'v_p130_td': 'Not enough future data for 130 trading days forward', 'p_p180_td': 'Not enough future data for 180 trading days forward', 'v_p180_td': 'Not enough future data for 180 trading days forward', 'p_p245_td': 'Not enough future data for 245 trading days forward', 'v_p245_td': 'Not enough future data for 245 trading days forward'}, 2688: {'p_p90_td': 'Not enough future data for 90 trading days forward', 'v_p90_td': 'Not enough future data for 90 tradin

In [ ]:
import csv

def write_error_dict_to_csv(error_dict, filename='error_report.csv'):
    # Open CSV file for writing
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['ticker', 'row_index', 'error_type', 'error_message']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        # Loop over the error dict
        for ticker, ticker_errors in error_dict.items():
            if isinstance(ticker_errors, dict):
                for row_idx, errors in ticker_errors.items():
                    # If errors is a dict of field:message
                    if isinstance(errors, dict):
                        for error_type, error_msg in errors.items():
                            writer.writerow({
                                'ticker': ticker,
                                'row_index': row_idx,
                                'error_type': error_type,
                                'error_message': error_msg
                            })
                    else:
                        # In case errors is just a single message
                        writer.writerow({
                            'ticker': ticker,
                            'row_index': row_idx,
                            'error_type': 'error',
                            'error_message': errors
                        })
            else:
                # If ticker_errors is just a string or not a dict
                writer.writerow({
                    'ticker': ticker,
                    'row_index': '',
                    'error_type': 'error',
                    'error_message': ticker_errors
                })

# Usage:
write_error_dict_to_csv(error_dict, filename='d3_yf_errors_full_flat.csv')


In [ ]:
import csv

def write_first_error_per_ticker(error_dict, df, filename='error_first_per_ticker.csv'):
    # Open CSV file for writing
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['ticker', 'company_name', 'row_index', 'filing_date', 'trade_date', 'error_type', 'error_message']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for ticker, ticker_errors in error_dict.items():
            if isinstance(ticker_errors, dict) and len(ticker_errors) > 0:
                # Get the first row index (sorted)
                first_idx = sorted(ticker_errors.keys())[0]
                errors = ticker_errors[first_idx]
                # Get company name, filing_date, trade_date from the DataFrame for that ticker and index
                try:
                    row = df.loc[first_idx]
                    company_name = row.get('company_name', '')
                    filing_date = row.get('filing_date', '')
                    trade_date = row.get('trade_date', '')
                except Exception:
                    # fallback: try to get any row for the ticker
                    row = df[df['ticker'] == ticker].iloc[0]
                    company_name = row.get('company_name', '')
                    filing_date = row.get('filing_date', '')
                    trade_date = row.get('trade_date', '')

                # For all error types in this first error, print each as a column
                if isinstance(errors, dict):
                    # Get the first error only (if multiple)
                    error_type, error_msg = list(errors.items())[0]
                else:
                    error_type, error_msg = 'error', errors
                writer.writerow({
                    'ticker': ticker,
                    'company_name': company_name,
                    'row_index': first_idx,
                    'filing_date': filing_date,
                    'trade_date': trade_date,
                    'error_type': error_type,
                    'error_message': error_msg
                })
            else:
                # ticker_errors is just a string or empty
                writer.writerow({
                    'ticker': ticker,
                    'company_name': '',
                    'row_index': '',
                    'filing_date': '',
                    'trade_date': '',
                    'error_type': 'error',
                    'error_message': ticker_errors
                })

# Usage example:
write_first_error_per_ticker(error_dict, oip_w_prices, filename='d3_yf_errors_by_ticker_wdata.csv')
#FILTER:
#in excel by: error_type - only filing_price, and at the same time: filing_date NOT 2025 (those j dont have lookfwds beyond august duh)
#reduces down to ~2000 missing... womp womp


In [ ]:
#1) export error_dict as failed tickers dict to xl for manual check?
#2) manual url retrieval? or llm based agent search to get correct investing.com urls
#3) run script to get all investing.com data for all missing (from yahoofinance) (but valid) stocks
#4) run script to pull from those many downloaded csvs to a df
#5) run an EQUIVALENT BUT MODIFIED add_prices_to_oip_precise for these new data, 
    #5b) merge this df with the main stock df

#6) run filtering algo(s) - zero vol, zero price, ADDV, min vol, min $ flow etc
#7) run some viz to look at included vs excluded stocks
#7.5) add returns computation
#7.6) add trade-to-file or trade-to-mebuy delay to df (feature)
#7.7) add trade-to-mebuy price change to df (feature)

#8) fit model - RF? some other time-series vecto autoreg? dynamic hold dur pred? return trade +%^ confidence? <-useful?
    #8a) tune model? ver vs val/train-test loss viz?
    #8b) R^2, F1, recall/precision metric sorting
    #8c) model selection
#9) pull out-of-training set OI data
    #9b) format like training data dfs
#10) pass into model, predict returns

#11) simulate trading strategy
    #skip negative predicted returns (duh)
    #skip small pos pred returns? ie threshold if pred to return less than 1% over the pred ideal hold duration
    #incl ADDV restriction - ie: give maximum trade size - cant go over say 1% (or .5%) of avg daily $ vol
    #dynamic buy based on +^% confidence (if part of model)
    #add buy/hold checkins to RF/conditional model - 
        #if after 1 day of hold it goes down x%, how likely to still finish in the green?